In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the lenna.jpg image
image = cv2.imread('lenna.jpg', cv2.IMREAD_GRAYSCALE)

# Check if the image is loaded successfully
if image is None:
    print("Error: Could not load image.")
else:
    print("Image loaded successfully.")


In [ ]:
height, width = image.shape
print('Original height:', height)
print('Original width:', width)

# Crop the image to 256x255
cropped_image = image[:256, :]

# Pad the image to 256x256
padded_image = np.pad(cropped_image, ((0, 0), (0, 1)), 'constant', constant_values=0)

# Verify the new dimensions
new_height, new_width = padded_image.shape
print('New height:', new_height)
print('New width:', new_width)

# Proceed with the DCT transformation on the 256x256 image
region_size = 32

num_rows = new_height // region_size
num_cols = new_width // region_size

print(num_rows, num_cols)

dct_coefficients = []

for i in range(num_rows):
    for j in range(num_cols):
        region = padded_image[i * region_size : (i + 1) * region_size, j * region_size : (j + 1) * region_size]
        dct = cv2.dct(np.float32(region))
        dct_coefficients.append(dct)

dct_coefficients = np.array(dct_coefficients)
print('dct_coefficients: ', len(dct_coefficients))


THRESHOLD

In [ ]:

# Define the percentage of DCT coefficients to keep (p)
p = 50  # You can adjust this value as needed

# Initialize a list to store the selected DCT coefficients
selected_coefficients = []

# Iterate through each region and select DCT coefficients using the zone method
for dct in dct_coefficients:
    # Calculate the threshold value based on the percentage p
    threshold = np.percentile(np.abs(dct), 100 - p)
    
    # Apply the zone method to select coefficients above the threshold
    selected_dct = np.where(np.abs(dct) >= threshold, dct, 0)
    
    # Append the selected coefficients to the list
    selected_coefficients.append(selected_dct)

# Check the number of regions with selected DCT coefficients
print('Number of selected DCT coefficient regions:', len(selected_coefficients))

# Verify the shape of the first region to ensure correctness
print('Shape of the first selected DCT coefficient region:', selected_coefficients[0].shape)

# Check the number of non-zero coefficients in the first region to ensure correctness
print('Number of non-zero coefficients in the first selected DCT coefficient region:', np.count_nonzero(selected_coefficients[0]))




ZONE


In [ ]:
# Define the percentage of DCT coefficients to keep (p)
p = 5  # You can adjust this value as needed

# Initialize a list to store the selected DCT coefficients
selected_coefficients = []


# Iterate through each region and select DCT coefficients using the zone method
for dct in dct_coefficients:
    # Calculate the total number of coefficients
    total_coefficients = dct.size
    
    # Calculate the number of coefficients to keep based on p%
    num_coefficients_to_keep = int(total_coefficients * p / 100)
    
    # Flatten the DCT coefficients and sort them by magnitude
    flat_dct = dct.flatten()
    sorted_indices = np.argsort(np.abs(flat_dct))[::-1]
    
    # Initialize a mask to select the top p% coefficients
    mask = np.zeros_like(flat_dct, dtype=bool)
    mask[sorted_indices[:num_coefficients_to_keep]] = True
    
    # Reshape the mask back to the original DCT coefficient shape
    mask = mask.reshape(dct.shape)
    
    # Apply the mask to select the top p% coefficients
    selected_dct = np.where(mask, dct, 0)
    
    # Append the selected coefficients to the list
    selected_coefficients.append(selected_dct)

# Check the number of regions with selected DCT coefficientsA
print('Number of selected DCT coefficient regions:', len(selected_coefficients))

# Verify the shape of the first region to ensure correctness
print('Shape of the first selected DCT coefficient region:', selected_coefficients[0].shape)

# Check the number of non-zero coefficients in the first region to ensure correctness
print('Number of non-zero coefficients in the first selected DCT coefficient region:', np.count_nonzero(selected_coefficients[0]))

In [ ]:
# Initialize an empty array to store the reconstructed image
reconstructed_image = np.zeros_like(padded_image)

# Initialize counters to keep track of the current region index
region_index = 0

# Iterate through each region and reconstruct the image using the selected DCT coefficients
for i in range(num_rows):
    for j in range(num_cols):
        # Get the selected DCT coefficients for the current region
        selected_dct = selected_coefficients[region_index]
        
        # Apply the inverse 2D-DCT to reconstruct the region
        reconstructed_region = cv2.idct(selected_dct)
        
        # Place the reconstructed region into the corresponding position in the reconstructed image
        reconstructed_image[i * region_size : (i + 1) * region_size, j * region_size : (j + 1) * region_size] = reconstructed_region
        
        # Increment the region index
        region_index += 1


In [ ]:
import matplotlib.pyplot as plt

# Display the original and reconstructed images side by side for visual inspection
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(padded_image, cmap='gray')
plt.title('Original Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(reconstructed_image, cmap='gray')
plt.title('Reconstructed Image')
plt.axis('off')
plt.show()

# Calculate the Mean Squared Error (MSE) between the original and reconstructed images
mse = np.mean((padded_image - reconstructed_image) ** 2)
print('Mean Squared Error (MSE):', mse)

THRESHOLD

In [ ]:
# Define the range of percentages to keep
p_values = list(range(5, 55, 5))

# Initialize a list to store the MSE values for each p
mse_values = []

for p in p_values:
    # Initialize a list to store the selected DCT coefficients
    selected_coefficients = []

    # Iterate through each region and select DCT coefficients using the zone method
    for dct in dct_coefficients:
        # Calculate the threshold value based on the percentage p
        threshold = np.percentile(np.abs(dct), 100 - p)

        # Apply the zone method to select coefficients above the threshold
        selected_dct = np.where(np.abs(dct) >= threshold, dct, 0)

        # Append the selected coefficients to the list
        selected_coefficients.append(selected_dct)

    # Initialize an empty array to store the reconstructed image
    reconstructed_image = np.zeros_like(padded_image)

    # Initialize counters to keep track of the current region index
    region_index = 0

    # Iterate through each region and reconstruct the image using the selected DCT coefficients
    for i in range(num_rows):
        for j in range(num_cols):
            # Get the selected DCT coefficients for the current region
            selected_dct = selected_coefficients[region_index]

            # Apply the inverse 2D-DCT to reconstruct the region
            reconstructed_region = cv2.idct(selected_dct)

            # Place the reconstructed region into the corresponding position in the reconstructed image
            reconstructed_image[i * region_size : (i + 1) * region_size, j * region_size : (j + 1) * region_size] = reconstructed_region

            # Increment the region index
            region_index += 1

    # Calculate the Mean Squared Error (MSE) between the original and reconstructed images
    mse = np.mean((padded_image - reconstructed_image) ** 2)
    mse_values.append(mse)

# Plot the relationship between p and MSE
plt.figure(figsize=(10, 5))
plt.plot(p_values, mse_values, marker='o')
plt.xlabel('Percentage of DCT Coefficients (p)')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('Effect of DCT Coefficient Percentage on Reconstruction Error')
plt.grid(True)
plt.show()

# Display the MSE values
for p, mse in zip(p_values, mse_values):
    print(f'p = {p}%, MSE = {mse:.2f}')


ZONE

In [ ]:
# Define the range of percentages to keep
p_values = list(range(5, 55, 5))

# Initialize a list to store the MSE values for each p
mse_values = []

for p in p_values:
    # Initialize a list to store the selected DCT coefficients
    selected_coefficients = []

    # Iterate through each region and select DCT coefficients using the zone method
    for dct in dct_coefficients:
        # Calculate the total number of coefficients
        total_coefficients = dct.size
        
        # Calculate the number of coefficients to keep based on p%
        num_coefficients_to_keep = int(total_coefficients * p / 100)
        
        # Flatten the DCT coefficients and sort them by magnitude
        flat_dct = dct.flatten()
        sorted_indices = np.argsort(np.abs(flat_dct))[::-1]
        
        # Initialize a mask to select the top p% coefficients
        mask = np.zeros_like(flat_dct, dtype=bool)
        mask[sorted_indices[:num_coefficients_to_keep]] = True
        
        # Reshape the mask back to the original DCT coefficient shape
        mask = mask.reshape(dct.shape)
        
        # Apply the mask to select the top p% coefficients
        selected_dct = np.where(mask, dct, 0)
        
        # Append the selected coefficients to the list
        selected_coefficients.append(selected_dct)


    # Initialize an empty array to store the reconstructed image
    reconstructed_image = np.zeros_like(padded_image)

    # Initialize counters to keep track of the current region index
    region_index = 0

    # Iterate through each region and reconstruct the image using the selected DCT coefficients
    for i in range(num_rows):
        for j in range(num_cols):
            # Get the selected DCT coefficients for the current region
            selected_dct = selected_coefficients[region_index]

            # Apply the inverse 2D-DCT to reconstruct the region
            reconstructed_region = cv2.idct(selected_dct)

            # Place the reconstructed region into the corresponding position in the reconstructed image
            reconstructed_image[i * region_size : (i + 1) * region_size, j * region_size : (j + 1) * region_size] = reconstructed_region

            # Increment the region index
            region_index += 1

    # Calculate the Mean Squared Error (MSE) between the original and reconstructed images
    mse = np.mean((padded_image - reconstructed_image) ** 2)
    mse_values.append(mse)

# Plot the relationship between p and MSE
plt.figure(figsize=(10, 5))
plt.plot(p_values, mse_values, marker='o')
plt.xlabel('Percentage of DCT Coefficients (p)')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('Effect of DCT Coefficient Percentage on Reconstruction Error')
plt.grid(True)
plt.show()

# Display the MSE values
for p, mse in zip(p_values, mse_values):
    print(f'p = {p}%, MSE = {mse:.2f}')
